<a href="https://colab.research.google.com/github/songmac/2023-Sesac-Lecture-and-Project/blob/master/231020_%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%B6%84%EC%84%9D_%ED%82%A4%EC%9B%8C%EB%93%9C_%EC%B6%94%EC%B6%9C(TF_IDF)_%EB%84%A4%EC%9D%B4%EB%B2%84_%EB%89%B4%EC%8A%A4%2C_%EC%98%81%ED%99%94%EB%A6%AC%EB%B7%B0_%EA%B0%90%EC%A0%95%EB%B6%84%EC%84%9D(solving).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TF-IDF를 사용한 키워드 추출

In [ ]:
import requests
from bs4 import BeautifulSoup

url = "https://n.news.naver.com/mnews/article/018/0005596218?sid=101"
resp = requests.get(url)
soup = BeautifulSoup(resp.content, 'html.parser')
content = soup.find(class_='go_trans _article_content').text.replace('\n',' ') #select문도 써보기

In [ ]:
content

' 기술혁신대전·경영혁신대회 통합 행사17~18일 이틀간 부산 벡스코서 열어중소기업 혁신 이끈 유공자에 포상혁신기업 전시·홍보하고 세미나 진행[이데일리 김경은 기자] 중소기업의 기술·경영 혁신 성과를 다루는 국내 최대 행사인 ‘2023 중소기업 기술·경영 혁신대전’이 오는 17~18일 부산 벡스코에서 열린다.    (사진=2023 중소기업 기술·경영 혁신대전 공식 누리집)15일 중소벤처기업부에 따르면 중소기업 기술·경영 혁신대전은 ‘혁신형 중소기업, 세상을 바꾸다’라는 주제로 개최된다. △탄소중립·ESG △글로벌 △연구개발(R&D) 혁신 △디지털 △기술보호 등 5대 중점분야별 기술·경영 혁신성과를 공유하고 확산할 예정이다. 기술·경영 혁신대전은 급변하는 기술·경영환경에 대응해 중소기업의 혁신을 지원하기 위해 열리는 행사다. 2000년부터 시작한 ‘중소기업 기술혁신 대전’과 2018년부터 개최한 ‘중소기업 경영혁신대회’를 통합해 규모를 확대했다. 각 행사는 그동안 서울에서 열렸으나 올해는 2030 부산 엑스포 유치 홍보를 지원하기 위해 부산으로 옮겨 개최한다. 이틀간 열리는 행사에서는 중소기업 혁신 유공 포상을 비롯해 △기술·기능 인재 경진대회 △인수합병(M&A) 및 상생 투자를 위한 IR 피칭 포럼 △스케일업 팁스 컨퍼런스 등 중소기업 혁신성장을 위한 각종 세미나 및 컨퍼런스를 진행한다. 본 행사인 ‘중소기업 혁신 유공 시상식’은 이영 중기부 장관이 참석해 기술·경영혁신으로 우수한 성과를 달성하고 국가 경제 발전에 이바지한 중소기업 및 관계 유공자를 시상할 예정이다. 포상은 훈장(3점), 포장(4점), 대통령표창(20점), 국무총리표창(30점), 장관표창(171점) 등 총 228점이 수여된다.행사장에는 △탄소중립·ESG △글로벌 △R&D혁신 △디지털 △테마정책관 총 5개의 구역으로 나눠 주관기관별 전시·홍보부스를 운영하고 혁신 성과를 홍보한다.이번 행사는 온·오프라인에서 누구나 무료로 참석 가능하다. 온라인 참여는 공식 유튜브 채널과 누리집에서 생중계된다. '

In [ ]:
# ! pip install konlpy

# 데이터 전처리

형태소 분석

In [ ]:
# 코모란
from konlpy.tag import Komoran
tokenizer = Komoran()

content_token = tokenizer.morphs(content)
print(content_token)

['기술', '혁신', '대전', '·', '경영', '혁신', '대회', '통합', '행사', '17', '~', '18', '일', '이틀', '간', '부산', '벡스코', '서', '열', '어', '중소기업', '혁신', '이끌', 'ㄴ', '유공자', '에', '포상', '혁신', '기업', '전시', '·', '홍보', '하', '고', '세미나', '진행', '[', '이데일리', '김경', '은', '기자', ']', '중소기업', '의', '기술', '·', '경영', '혁신', '성과', '를', '다루', '는', '국내', '최대', '행사', '이', 'ㄴ', '‘', '2023', '중소기업', '기술', '·', '경영', '혁신', '대전', '’', '이', '오', '는', '17', '~', '18', '일', '부산', '벡스코', '에서', '열리', 'ㄴ다', '.', '(', '사진', '=', '2023', '중소기업', '기술', '·', '경영', '혁신', '대전', '공식', '누리', '집', ')', '15', '일', '중소', '벤처기업', '부', '에', '따르', '면', '중소기업', '기술', '·', '경영', '혁신', '대전', '은', '‘', '혁신', '형', '중소기업', ',', '세상', '을', '바꾸', '다', '’', '이', '라는', '주제', '로', '개최', '되', 'ㄴ다', '.', '△', '탄소', '중립', '·', 'ESG', '△', '글로벌', '△', '연구개발', '(', 'R', '&', 'D', ')', '혁신', '△', '디지털', '△', '기술', '보호', '등', '5', '대', '중점', '분야', '별', '기술', '·', '경영', '혁신', '성과', '를', '공유', '하', '고', '확산', '하', 'ㄹ', '예정', '이', '다', '.', '기술', '·', '경영', '혁신', '대전', '은', '급변

한글자 제거

In [ ]:
token_remove = []
for token in content_token:
  if len(token)>1:
    token_remove.append(token)
print(token_remove)

['기술', '혁신', '대전', '경영', '혁신', '대회', '통합', '행사', '17', '18', '이틀', '부산', '벡스코', '중소기업', '혁신', '이끌', '유공자', '포상', '혁신', '기업', '전시', '홍보', '세미나', '진행', '이데일리', '김경', '기자', '중소기업', '기술', '경영', '혁신', '성과', '다루', '국내', '최대', '행사', '2023', '중소기업', '기술', '경영', '혁신', '대전', '17', '18', '부산', '벡스코', '에서', '열리', 'ㄴ다', '사진', '2023', '중소기업', '기술', '경영', '혁신', '대전', '공식', '누리', '15', '중소', '벤처기업', '따르', '중소기업', '기술', '경영', '혁신', '대전', '혁신', '중소기업', '세상', '바꾸', '라는', '주제', '개최', 'ㄴ다', '탄소', '중립', 'ESG', '글로벌', '연구개발', '혁신', '디지털', '기술', '보호', '중점', '분야', '기술', '경영', '혁신', '성과', '공유', '확산', '예정', '기술', '경영', '혁신', '대전', '급변', '기술', '경영', '환경', '대응', '중소기업', '혁신', '지원', '위하', '열리', '행사', '2000', '부터', '시작', '중소기업', '기술', '혁신', '대전', '2018', '부터', '개최', '중소기업', '경영', '혁신', '대회', '통합', '규모', '확대', '행사', '그동안', '서울', '에서', '열리', '으나', '올해', '2030', '부산', '엑스포', '유치', '홍보', '지원', '위하', '부산', '으로', '옮기', '개최', 'ㄴ다', '이틀', '열리', '행사', '에서', '중소기업', '혁신', '유공', '포상', '비롯', '기술', '기능', '인재', '경진', '대회', '인수', 

불용어처리

In [ ]:
stop_word = ['15', '17', '18','20', '30', '171', '228', '2000', '2018', '2023', '2030', 'ㄴ다', '에서']

token_list = []
for token in token_remove:
  if token not in stop_word:
    token_list.append(token)
print(token_list)

['기술', '혁신', '대전', '경영', '혁신', '대회', '통합', '행사', '이틀', '부산', '벡스코', '중소기업', '혁신', '이끌', '유공자', '포상', '혁신', '기업', '전시', '홍보', '세미나', '진행', '이데일리', '김경', '기자', '중소기업', '기술', '경영', '혁신', '성과', '다루', '국내', '최대', '행사', '중소기업', '기술', '경영', '혁신', '대전', '부산', '벡스코', '열리', '사진', '중소기업', '기술', '경영', '혁신', '대전', '공식', '누리', '중소', '벤처기업', '따르', '중소기업', '기술', '경영', '혁신', '대전', '혁신', '중소기업', '세상', '바꾸', '라는', '주제', '개최', '탄소', '중립', 'ESG', '글로벌', '연구개발', '혁신', '디지털', '기술', '보호', '중점', '분야', '기술', '경영', '혁신', '성과', '공유', '확산', '예정', '기술', '경영', '혁신', '대전', '급변', '기술', '경영', '환경', '대응', '중소기업', '혁신', '지원', '위하', '열리', '행사', '부터', '시작', '중소기업', '기술', '혁신', '대전', '부터', '개최', '중소기업', '경영', '혁신', '대회', '통합', '규모', '확대', '행사', '그동안', '서울', '열리', '으나', '올해', '부산', '엑스포', '유치', '홍보', '지원', '위하', '부산', '으로', '옮기', '개최', '이틀', '열리', '행사', '중소기업', '혁신', '유공', '포상', '비롯', '기술', '기능', '인재', '경진', '대회', '인수', '합병', '상생', '투자', '위하', 'IR', '피칭', '포럼', '스케일', '팁스', '컨퍼런스', '중소기업', '혁신', '성장', '위하', '각종', '세미나', '컨퍼런

In [ ]:
content_token = ' '.join(token_list) #문장처럼 만들어주고 추후 공백으로 토큰화 하기 위함
content_token

'기술 혁신 대전 경영 혁신 대회 통합 행사 이틀 부산 벡스코 중소기업 혁신 이끌 유공자 포상 혁신 기업 전시 홍보 세미나 진행 이데일리 김경 기자 중소기업 기술 경영 혁신 성과 다루 국내 최대 행사 중소기업 기술 경영 혁신 대전 부산 벡스코 열리 사진 중소기업 기술 경영 혁신 대전 공식 누리 중소 벤처기업 따르 중소기업 기술 경영 혁신 대전 혁신 중소기업 세상 바꾸 라는 주제 개최 탄소 중립 ESG 글로벌 연구개발 혁신 디지털 기술 보호 중점 분야 기술 경영 혁신 성과 공유 확산 예정 기술 경영 혁신 대전 급변 기술 경영 환경 대응 중소기업 혁신 지원 위하 열리 행사 부터 시작 중소기업 기술 혁신 대전 부터 개최 중소기업 경영 혁신 대회 통합 규모 확대 행사 그동안 서울 열리 으나 올해 부산 엑스포 유치 홍보 지원 위하 부산 으로 옮기 개최 이틀 열리 행사 중소기업 혁신 유공 포상 비롯 기술 기능 인재 경진 대회 인수 합병 상생 투자 위하 IR 피칭 포럼 스케일 팁스 컨퍼런스 중소기업 혁신 성장 위하 각종 세미나 컨퍼런스 진행 행사 중소기업 혁신 유공 시상식 이영 중기 장관 참석 기술 경영 혁신 으로 우수 성과 달성 국가 경제 발전 이바지 중소기업 관계 유공자 시상 예정 포상 훈장 포장 대통령 표창 국무총리 표창 장관 표창 수여 행사 탄소 중립 ESG 글로벌 혁신 디지털 테마 정책관 구역 으로 나누 주관 기관 전시 홍보 부스 운영 혁신 성과 홍보 이번 행사 오프라인 누구 무료 참석 가능 온라인 참여 공식 유튜브 채널 누리 중계'

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer()
tfidf_v = tfidf_vect.fit_transform([content_token]) #단어셋(문서) fitting

keyword = tfidf_v.tocoo() #등장하지 않는 단어를 표현하지 않음
# coo : coordinate 으로 0이 아닌 데이터만 별도의 배열에 저장하고, 그 데이터가 가리키는 행과 열의 위치를 별도의 배열에 저장하는 방식

In [ ]:
sorted_words = sorted(zip(keyword.col, keyword.data), key=lambda x:(x[1], x[0]), reverse=True)
sorted_words #(단어 인덱스, if_idf값) #큰 숫자부터 보이도록 sorted

[(115, 0.5946837453780289),
 (94, 0.36813755666258935),
 (21, 0.3398192830731594),
 (5, 0.2831827358942995),
 (114, 0.2265461887154396),
 (31, 0.1699096415365797),
 (116, 0.1132730943577198),
 (70, 0.1132730943577198),
 (62, 0.1132730943577198),
 (51, 0.1132730943577198),
 (43, 0.1132730943577198),
 (111, 0.08495482076828985),
 (109, 0.08495482076828985),
 (76, 0.08495482076828985),
 (33, 0.08495482076828985),
 (4, 0.08495482076828985),
 (105, 0.0566365471788599),
 (103, 0.0566365471788599),
 (102, 0.0566365471788599),
 (98, 0.0566365471788599),
 (97, 0.0566365471788599),
 (96, 0.0566365471788599),
 (92, 0.0566365471788599),
 (86, 0.0566365471788599),
 (85, 0.0566365471788599),
 (82, 0.0566365471788599),
 (72, 0.0566365471788599),
 (71, 0.0566365471788599),
 (63, 0.0566365471788599),
 (53, 0.0566365471788599),
 (45, 0.0566365471788599),
 (40, 0.0566365471788599),
 (34, 0.0566365471788599),
 (27, 0.0566365471788599),
 (17, 0.0566365471788599),
 (8, 0.0566365471788599),
 (0, 0.0566365471

In [ ]:
feature_name = tfidf_vect.get_feature_names_out() #단어 이름 출력
feature_name

array(['esg', 'ir', '가능', '각종', '개최', '경영', '경제', '경진', '공식', '공유', '관계',
       '구역', '국가', '국내', '국무총리', '규모', '그동안', '글로벌', '급변', '기관', '기능',
       '기술', '기업', '기자', '김경', '나누', '누구', '누리', '다루', '달성', '대응', '대전',
       '대통령', '대회', '디지털', '따르', '라는', '무료', '바꾸', '발전', '벡스코', '벤처기업',
       '보호', '부산', '부스', '부터', '분야', '비롯', '사진', '상생', '서울', '성과', '성장',
       '세미나', '세상', '수여', '스케일', '시상', '시상식', '시작', '엑스포', '연구개발', '열리',
       '예정', '오프라인', '온라인', '올해', '옮기', '우수', '운영', '위하', '유공', '유공자',
       '유치', '유튜브', '으나', '으로', '이끌', '이데일리', '이바지', '이번', '이영', '이틀',
       '인수', '인재', '장관', '전시', '정책관', '주관', '주제', '중계', '중기', '중립', '중소',
       '중소기업', '중점', '지원', '진행', '참석', '참여', '채널', '최대', '컨퍼런스', '탄소',
       '테마', '통합', '투자', '팁스', '포럼', '포상', '포장', '표창', '피칭', '합병', '행사',
       '혁신', '홍보', '확대', '확산', '환경', '훈장'], dtype=object)

In [ ]:
# 핵심키워드 10개 추출
[(feature_name[i], score) for i,score in sorted_words[:10]] # score : tfidf 값

[('혁신', 0.5946837453780289),
 ('중소기업', 0.36813755666258935),
 ('기술', 0.3398192830731594),
 ('경영', 0.2831827358942995),
 ('행사', 0.2265461887154396),
 ('대전', 0.1699096415365797),
 ('홍보', 0.1132730943577198),
 ('위하', 0.1132730943577198),
 ('열리', 0.1132730943577198),
 ('성과', 0.1132730943577198)]

# 실습 2. 영화 리뷰에서 '긍정', '부정' 주요 키워드는 무엇인지 확인하고, 워드클라우드 그리기

작업 순서
- 1) 리뷰 데이터를 load -> data cleansing -> 토큰화 -> 불용어처리
- 2) TF-IDF로, 각 문장별 키워드 추출(한 문장에서 5개까지)하여 컬럼에 넣기
- 3) 긍정리뷰 추출(label =1) : keyword 컬럼에 있는 모든 단어를 집계
- 4) 워드 클라우드로 표현
- 5) 3,4 반복하면 부정리뷰 키워드 추출도 가능